In [19]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Optional

In [20]:
class PrivateState(TypedDict):
    a: str
    b: str


class MegaPrivate(TypedDict):
    secret: bool


class InputState(TypedDict):
    hello: str


class OutputState(TypedDict):
    bye: str


graph_builder = StateGraph(
    PrivateState,
    input_schema=InputState,
    output_schema=OutputState,
)

In [21]:
from this import d


def node_one(state: InputState) -> InputState:
    print("node_one received", state)
    return {
        "hello": "hello from node one",
    }


def node_two(state: PrivateState) -> PrivateState:
    print("node_two received", state)
    return {
        "a": "1",
    }


def node_three(state: PrivateState) -> PrivateState:
    print("node_three received", state)
    return {
        "b": "2",
    }


def node_four(state: PrivateState) -> OutputState:
    print("node_four received", state)
    return {
        "bye": "bye from node four",
    }


def node_five(state: PrivateState):
    return {"secret": True}


def node_six(state: MegaPrivate):
    print("MegaPrivate in node_six", state)

In [22]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)
graph_builder.add_node("node_five", node_five)
graph_builder.add_node("node_six", node_six)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", "node_four")
graph_builder.add_edge("node_four", "node_five")
graph_builder.add_edge("node_five", "node_six")
graph_builder.add_edge("node_six", END)

In [23]:
graph = graph_builder.compile()

# Better UI but requires internet connection
# graph
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
 +----------+  
 | node_one |  
 +----------+  
       *       
       *       
       *       
 +----------+  
 | node_two |  
 +----------+  
       *       
       *       
       *       
+------------+ 
| node_three | 
+------------+ 
       *       
       *       
       *       
+-----------+  
| node_four |  
+-----------+  
       *       
       *       
       *       
+-----------+  
| node_five |  
+-----------+  
       *       
       *       
       *       
 +----------+  
 | node_six |  
 +----------+  
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


In [24]:
a = {"hello": "123"}

graph.invoke(a)

node_one received {'hello': '123'}
node_two received {}
node_three received {'a': '1'}
node_four received {'a': '1', 'b': '2'}
MegaPrivate in node_six {'secret': True}


{'bye': 'bye from node four'}